In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

T = 1000

time = np.arange(0,T)
fn = np.sin(time/100)

noise = np.random.randn(len(time))/15
fn = fn+noise

plt.plot(time, fn)
plt.show()

In [ ]:
def prep_data_old(_tau):
    tau = _tau
    data = np.zeros((time.shape[0]-tau, tau))
    for a in range(len(time)-tau):
        data[a] = fn[a:a+tau]
    y = data[:,-1]
    X = data[:,:-1]

    split = len(X)//5
    X_train, X_test =  X[split:], X[:split]
    y_train, y_test = y[split:], y[:split]

    return X_train, X_test, y_train, y_test

In [ ]:
def get_model():
    model = keras.Sequential()
    model.add(layers.Embedding(input_dim=X_train.shape[1], output_dim=4))
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1))
    opt = tf.keras.optimizers.Adam(.1)
    model.compile(optimizer=opt, loss='mse', metrics=['mse', 'mae'])
    return model

get_model().summary()

In [ ]:
X_train, X_test, y_train, y_test = prep_data(20)
model = get_model()
model.fit(X_train, y_train, epochs=2, batch_size=16, verbose=0)
model.evaluate(X_test, y_test)
plt.plot(y_train, label='truth')
plt.plot(model.predict(X_train), label='predict')
#plt.title(f'tau: {taus}')
plt.legend()
plt.show()

In [ ]:
def prep_data(_tau):
    tau = _tau
    data = np.zeros((time.shape[0]-tau, tau*2))
    for a in range(len(time)-tau):
        data[a] = np.append(fn[a:a+tau], fn[a+1:a+tau+1])
        
    y = data[:,-tau:]
    X = data[:,:-tau]

    split = len(X)//5
    X_train, X_test =  X[split:], X[:split]
    y_train, y_test = y[split:], y[:split]

    return X_train, X_test, y_train, y_test